# Interactive Risk Dashboard

This notebook demonstrates the interactive risk dashboard with:
- **Exceedance curve** with ES 95% highlighted
- **Clickable points** to see DAG snapshots
- **Failed nodes** highlighted in red
- **Failure propagation** shown with red edges
- **Persistent layout** across snapshots

## 1. Setup

In [1]:
from riskdag import RiskDAG, RiskNode, LatentRiskNode
from riskdag.visualization import create_interactive_dashboard
from scipy import stats

print("Imports successful!")

from IPython.display import display, HTML

# Override Jupyter's scrollbar
display(HTML("""
<style>
.jp-OutputArea-output, .output_scroll {
    max-height: none !important;
    overflow: visible !important;
}
</style>
"""))

Imports successful!


## 2. Create Example Pipeline

In [2]:
with RiskDAG('etl_pipeline', time_scale='hour') as dag:
    # Latent risks
    cloud = LatentRiskNode('cloud', p_fail=0.01, 
                         loss_dist=stats.norm(5000, 1000),
                         user_time_scale='day')
    database = LatentRiskNode('database', p_fail=0.02,
                              loss_dist=stats.norm(10000, 2000),
                              user_time_scale='day')
    network = LatentRiskNode('network', p_fail=0.015,
                             loss_dist=stats.norm(3000, 500),
                             user_time_scale='day')
    
    # Tasks
    extract = RiskNode('extract', p_fail=0.01, loss_dist=stats.norm(500, 100))
    transform = RiskNode('transform', p_fail=0.05, loss_dist=stats.norm(1000, 200))
    validate = RiskNode('validate', p_fail=0.02, loss_dist=stats.norm(1500, 300))
    load = RiskNode('load', p_fail=0.02, loss_dist=stats.norm(800, 150))
    
    # Relationships
    cloud @ database | 0.7
    database >> extract
    network @ load | 0.7
    extract >> transform >> validate >> load

print(f"Created: {dag}")

Created: RiskDAG('etl_pipeline', nodes=7, latent=3)


## 3. Run Monte Carlo Simulation

In [3]:
# Run 1000 simulations
results = dag.run_monte_carlo(10000, seed=42)

print("Simulation complete!")
print(f"  Mean Loss: ${results.summary_statistics()['mean_loss']:,.0f}")
print(f"  VaR 95%:   ${results.summary_statistics()['var_95']:,.0f}")
print(f"  ES 95%:    ${results.summary_statistics()['es_95']:,.0f}")

Simulation complete!
  Mean Loss: $287
  VaR 95%:   $2,995
  ES 95%:    $3,901


## 4. Create Interactive Dashboard

The dashboard shows:
- **Pink region** = ES 95% tail on exceedance curve
- **Top contributors** to Expected Shortfall
- **DAG snapshots** for specific simulations

In [4]:
# Create dashboard
dashboard = create_interactive_dashboard(dag, results, es_percentile=0.98)

# Display
widget = dashboard.create_dashboard()
display(widget)

## 5. Explore Specific Simulations

Click on the exceedance curve above, or run cells below to see specific scenarios.

### Low-Loss Scenario (Typical)

In [5]:
# Show a typical low-loss simulation
dashboard.show_simulation(6446)

### High-Loss Scenario (ES Tail)

In [6]:
# Show a high-loss simulation in the ES tail
dashboard.show_simulation(950)

### Worst Case

In [7]:
# Find and show the worst-case simulation
worst_idx = results.total_losses.argmax()
print(f"Worst case: Simulation #{worst_idx} with loss ${results.total_losses[worst_idx]:,.0f}")

dashboard.show_simulation(worst_idx)

Worst case: Simulation #8971 with loss $20,515


## Understanding the Visualization

### Node Colors
- **White with green border** = Task that didn't fail
- **Pink with red border** = Task that failed (shows cost)
- **Gray with dashed border** = Latent risk that didn't trigger
- **Orange with thick border** = Latent risk that triggered (shows cost)

### Edge Colors
- **Gray** = No failure propagation
- **Red (thick)** = Failure propagated through this edge
- **Dashed** = Latent risk connection

### Labels
- ✓ = Node OK
- ❌ = Task failed
- ⚡ = Latent risk triggered
- ○ = Latent risk dormant

### Layout
- Drag nodes to rearrange
- Layout persists when viewing other simulations!
- Makes it easy to compare different scenarios